In [9]:
import os
import numpy
import json

# Load Results and Errors

In [10]:
# Load Sample Prediction Tracks
TRACK_DIR = os.path.join(os.getcwd(), 'tracks/predicted_tracks_mini.json')
with open(TRACK_DIR, "r") as file:
    pred_json = json.load(file)

# Load Track Evaluation Errors
ERRORS_DIR = os.path.join(os.getcwd(), 'results/predicted_tracks_mini/errors.json')
with open(ERRORS_DIR, "r") as file:
    errors_json = json.load(file)


# How to Index Scenes in Error File

In [11]:
# Define Parameters
class_id = 'car'
thresh = 'thr_0.5231'
scene_token ='e233467e827140efa4b42d2b4c435855'

scene_id_errors = errors_json[class_id][thresh][scene_token]
print("Num SWITCH: ", scene_id_errors['num_switch'])
print("Num MISS: ", scene_id_errors['num_miss'])
print("Num FP: ", scene_id_errors['num_fp'])
print("Num of Frames in Scene: ", len(scene_id_errors['frame_errors']))

print("\n")

print("Errors in the 13th scene: \n")
print(json.dumps(scene_id_errors['frame_errors'][13], indent=4))

Num SWITCH:  1
Num MISS:  17
Num FP:  4
Num of Frames in Scene:  40


Errors in the 13th scene: 

{
    "SWITCH": [
        {
            "type": "SWITCH",
            "object_id": "bb0f40c613904cdcb3299a73e105c3c4",
            "hypothesis_id": 2230.0,
            "distance": 0.4180664422940312,
            "gt": {
                "num_pts": 1,
                "size": [
                    1.711,
                    4.047,
                    1.637
                ],
                "rotation": [
                    0.9055697244684551,
                    0.0,
                    0.0,
                    0.4241974470999635
                ],
                "velocity": [
                    5.277298468594521,
                    6.5086014281926525
                ],
                "translation": [
                    1330.821,
                    928.003,
                    1.53
                ],
                "ego_translation": [
                    9.490090139989661,
          

# Define Object of Interest

In [12]:
sample_id = '050b2d9c4bb647b7ab4ee94bdb538e1d'
object_id = 'bb0f40c613904cdcb3299a73e105c3c4'
hypothesis_id = int(scene_id_errors['frame_errors'][13]['SWITCH'][0]["hypothesis_id"])
print("Hypothesis:", hypothesis_id)

Hypothesis: 2230


# Find Prediction in Results File

In [13]:
sample_pred = pred_json['results'][sample_id]

# Find Hypothesis
obj_info = None
found_obj = False
for pred in sample_pred:
    if pred['tracking_id'] == hypothesis_id:
        obj_info = pred
        found_obj = True
        break

if found_obj:
    print(json.dumps(obj_info, indent=4))
else:
    print("Could not find prediction")

{
    "sample_token": "050b2d9c4bb647b7ab4ee94bdb538e1d",
    "translation": [
        1330.5415968726434,
        928.3139878491701,
        1.5523162465932605
    ],
    "size": [
        1.9228668212890625,
        4.482692718505859,
        1.6988316774368286
    ],
    "rotation": [
        -0.904163604642223,
        -0.007150067158249172,
        0.03588519925603187,
        -0.42561638250233036
    ],
    "velocity": [
        6.20677207288962,
        7.370825961635694
    ],
    "tracking_name": "car",
    "tracking_score": 0.5239403247833252,
    "tracking_id": 2230,
    "attribute_name": "vehicle.moving"
}


# Find Ground Truth

In [14]:
from nuscenes.nuscenes import NuScenes
nusc = NuScenes(version='v1.0-mini', verbose=True, dataroot='dataset/nuscenes')

Loading NuScenes tables for version v1.0-mini...
Loading table: sample_annotation.json
Loading table: ego_pose.json
Loading table: sample_data.json
Loading table: category.json
Loading table: attribute.json
Loading table: visibility.json
Loading table: instance.json
Loading table: sensor.json
Loading table: calibrated_sensor.json
Loading table: log.json
Loading table: scene.json
Loading table: sample.json
Loading table: map.json
Loading nuScenes-lidarseg...
Loading table: lidarseg.json
Loading table: lidarseg.json
Loading nuScenes-panoptic...
Loading table: panoptic.json
Loading table: panoptic.json
32 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
404 lidarseg,
404 panoptic,
Done loading in 0.262 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


In [15]:
# The instance token gets you the object in every frame it is annotated in
gt_instance = nusc.get('instance', object_id)
print(json.dumps(gt_instance, indent=4))

{
    "token": "bb0f40c613904cdcb3299a73e105c3c4",
    "category_token": "fd69059b62a3469fbaef25340c0eab7f",
    "nbr_annotations": 31,
    "first_annotation_token": "fedb34281be246299448d0647fb3b2df",
    "last_annotation_token": "9527523d07dc4794a52d70e8273ad199"
}


In [16]:
# Get the ground truth for the prediction at the specific frame
gt_sample = nusc.get('sample', sample_id)
gt_anno = None
# Iterate through the annotations in the sample frame
for anno in gt_sample['anns']:
    nusc_anno = nusc.get('sample_annotation', anno)
    if nusc_anno['instance_token'] == object_id:
        gt_anno = nusc_anno
print(json.dumps(gt_anno, indent=4))

{
    "token": "8c273f6c96ac4a8687cd6e75f718db70",
    "sample_token": "050b2d9c4bb647b7ab4ee94bdb538e1d",
    "instance_token": "bb0f40c613904cdcb3299a73e105c3c4",
    "visibility_token": "1",
    "attribute_tokens": [
        "cb5118da1ab342aa947717dc53544259"
    ],
    "translation": [
        1330.821,
        928.003,
        1.53
    ],
    "size": [
        1.711,
        4.047,
        1.637
    ],
    "rotation": [
        0.9055697244684551,
        0.0,
        0.0,
        0.4241974470999635
    ],
    "prev": "5de300a14552416897a9e22faca3be68",
    "next": "02274f020deb45828664a49631fe3540",
    "num_lidar_pts": 1,
    "num_radar_pts": 0,
    "category_name": "vehicle.car"
}
